In [77]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.estimators import H2ODeepLearningEstimator
from h2o.estimators import H2OSupportVectorMachineEstimator
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators import H2ONaiveBayesEstimator
from h2o.estimators import H2ODeepLearningEstimator
from h2o.estimators import H2OGeneralizedLinearEstimator
from h2o.estimators import H2ORandomForestEstimator

In [78]:
h2o.init(ip="localhost",port=54321)
m_models = {
    "bayes":H2ONaiveBayesEstimator(),
    "glm":H2OGeneralizedLinearEstimator(nfolds = 4),
    "rf":H2ORandomForestEstimator(nfolds = 4),
    "gbm":H2OGradientBoostingEstimator(nfolds=4),
    #"svm":H2OSupportVectorMachineEstimator(),
    "xgboost":H2OXGBoostEstimator(nfolds=4),
    "ann":H2ODeepLearningEstimator(nfolds=4)
}

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,39 days 17 hours 45 mins
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,7 months and 7 days !!!
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,19.92 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [79]:
df_raw = pd.read_csv('./pcap.csv')
df_raw = df_raw[df_raw['ip_version']==4.0]
del df_raw['ts']
del df_raw['pcap_row_id']
del df_raw['ip_src']
del df_raw['ip_dst']

del df_raw['ip_version']
del df_raw['ip_ihl']
del df_raw['ip_frag']
del df_raw['tcp_reserved']
del df_raw['tcp_urgptr']

del df_raw['icmp_nexthopmtu']
del df_raw['icmp_reserved']
del df_raw['icmp_code']
del df_raw['icmp_type']
del df_raw['icmp_length']

del df_raw['ip_chksum']
del df_raw['tcp_chksum']
del df_raw['udp_chksum']
del df_raw['icmp_chksum']

df_raw = df_raw.fillna(0)

df_raw['detailed_label'].value_counts()

/data/env/ai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


PartOfAHorizontalPortScan    145719
Benign                        19279
C&C                           14936
DDoS                          14394
Attack                         5962
C&C-Torii                        30
C&C-FileDownload                  4
FileDownload                      2
Name: detailed_label, dtype: int64

In [80]:
df_input = df_raw[(df_raw['detailed_label']=="Attack")|(df_raw['detailed_label']=="Benign")|(df_raw['detailed_label']=="C&C")|(df_raw['detailed_label']=="DDoS")]
df_input['detailed_label'].value_counts()

Benign    19279
C&C       14936
DDoS      14394
Attack     5962
Name: detailed_label, dtype: int64

In [81]:
X_train, X_test, Y_train, Y_test = train_test_split(df_input, df_input['detailed_label'], test_size=0.20)
X_train

,detailed_label,ip_tos,ip_len,ip_id,ip_flags,ip_ttl,ip_proto,tcp_sport,tcp_dport,tcp_seq,tcp_ack,tcp_dataofs,tcp_flags,tcp_window,udp_sport,udp_dport,udp_len,icmp_unused
163371,C&C,0.0,60.0,1505.0,DF,64.0,6.0,37008.0,50.0,3.268336e+09,0.0,10.0,S,29200.0,0.0,0.0,0.0,0
161133,Benign,184.0,76.0,13993.0,DF,64.0,17.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0.0,123.0,123.0,56.0,0
109291,Attack,0.0,60.0,41577.0,DF,64.0,6.0,55169.0,22.0,2.815201e+09,0.0,10.0,S,29200.0,0.0,0.0,0.0,0
178417,C&C,0.0,60.0,42522.0,DF,64.0,6.0,53650.0,6667.0,2.928275e+09,0.0,10.0,S,29200.0,0.0,0.0,0.0,0
184978,DDoS,0.0,1064.0,65279.0,0,255.0,6.0,65279.0,80.0,4.278190e+09,0.0,5.0,SPAU,65279.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180996,DDoS,0.0,1064.0,65279.0,0,255.0,6.0,65279.0,80.0,4.278190e+09,0.0,5.0,SPAU,65279.0,0.0,0.0,0.0,0
192951,DDoS,0.0,1064.0,65279.0,0,255.0,6.0,65279.0,80.0,4.278190e+09,0.0,5.0,SPAU,65279.0,0.0,0.0,0.0,0
170227,Benign,184.0,76.0,34818.0,DF,64.0,17.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0.0,123.0,123.0,56.0,0
162387,Benign,184.0,76.0,17108.0,DF,64.0,17.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0.0,123.0,123.0,56.0,0


In [82]:
def Train(df_all):
    df_h2o = h2o.H2OFrame(df_all)
    df_train, df_valid = df_h2o.split_frame(ratios=[0.85], seed=1234)
    x = df_train.columns
    y = "detailed_label"
    x.remove(y)
    df_train[y] = df_train[y].asfactor()
    df_valid[y] = df_valid[y].asfactor()
    for key in m_models:
        print("begin train ",key)
        m_models[key].train(x=x, y=y,training_frame=df_train,validation_frame=df_valid)
        print("end train ",key)

def Predict( df_all ):
    df_h2o = h2o.H2OFrame(df_all)
    x = df_h2o.columns
    y = "detailed_label"
    x.remove(y)
    df_h2o[y] = df_h2o[y].asfactor()
    ret = {}
    for key in m_models:
        pred = m_models[key].predict(df_h2o).as_data_frame()
        y_true = df_all[y].to_list()
        y_pred = pred['predict']
        ret[key] = {}
        ret[key]['y_pred'] = y_pred.to_list()
        ret[key]['y_true'] = y_true
    return ret

In [83]:
Train(X_train)
result = Predict(X_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
begin train  bayes
naivebayes Model Build progress: |

/data/env/ai/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:206: RuntimeWarning: Dropping bad and constant columns: [ip_flags]
  warnings.warn(mesg["message"], RuntimeWarning)


███████████████████████████████████████████████| (done) 100%
end train  bayes
begin train  glm
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
end train  glm
begin train  rf
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
end train  rf
begin train  gbm
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
end train  gbm
begin train  xgboost
xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%
end train  xgboost
begin train  ann
deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%
end train  ann
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
naivebayes prediction progress: |████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 1

In [84]:
from sklearn.metrics import classification_report
for model in result.keys():
    y_pred = result[model]['y_pred']
    y_true = result[model]['y_true']
    print(model)
    print(classification_report(y_true, y_pred, target_names=['Attack','Benign','C&C','DDoS'],digits=6))

bayes
              precision    recall  f1-score   support

      Attack   0.000000  0.000000  0.000000      1213
      Benign   1.000000  0.000515  0.001030      3883
         C&C   0.365516  1.000000  0.535352      2953
        DDoS   1.000000  0.987439  0.993680      2866

    accuracy                       0.530005     10915
   macro avg   0.591379  0.496989  0.382515     10915
weighted avg   0.717212  0.530005  0.406118     10915

glm
              precision    recall  f1-score   support

      Attack   0.606713  0.581204  0.593684      1213
      Benign   0.999659  0.754829  0.860161      3883
         C&C   0.699366  0.934304  0.799942      2953
        DDoS   0.996175  0.999651  0.997910      2866

    accuracy                       0.848374     10915
   macro avg   0.825478  0.817497  0.812924     10915
weighted avg   0.873833  0.848374  0.850425     10915

rf
              precision    recall  f1-score   support

      Attack   0.769231  0.931575  0.842655      1213
      Be

In [85]:
for key in m_models:
    print(key)
    varimp = m_models[key].varimp(use_pandas=True)
    display(varimp)

bayes


None

glm


,variable,relative_importance,scaled_importance,percentage
0,tcp_dataofs,4.374604,1.000000,0.208360
1,tcp_dport,3.230827,0.738541,0.153883
2,ip_ttl,3.051636,0.697580,0.145348
3,tcp_sport,2.990301,0.683559,0.142427
4,ip_len,2.373278,0.542513,0.113038
5,tcp_window,2.158777,0.493479,0.102822
6,tcp_ack,0.832507,0.190304,0.039652
7,udp_len,0.726125,0.165987,0.034585
8,tcp_flags.S,0.468800,0.107164,0.022329
9,ip_id,0.241599,0.055228,0.011507


rf


...

,variable,relative_importance,scaled_importance,percentage
0,tcp_flags,4.684175e+04,1.000000e+00,4.179446e-01
1,tcp_dport,2.997853e+04,6.399960e-01,2.674829e-01
2,tcp_window,1.556017e+04,3.321858e-01,1.388353e-01
3,tcp_dataofs,1.058667e+04,2.260093e-01,9.445939e-02
4,tcp_sport,7.870881e+03,1.680313e-01,7.022779e-02
5,ip_len,9.734735e+02,2.078218e-02,8.685800e-03
6,tcp_seq,1.342682e+02,2.866422e-03,1.198005e-03
7,ip_id,1.193441e+02,2.547815e-03,1.064846e-03
8,ip_ttl,1.063820e+01,2.271094e-04,9.491916e-05
9,tcp_ack,4.088312e-01,8.727925e-06,3.647789e-06


xgboost


,variable,relative_importance,scaled_importance,percentage
0,tcp_window,86176.437500,1.000000,4.919149e-01
1,tcp_dport,48856.207031,0.566932,2.788825e-01
2,tcp_flags.S,34856.519531,0.404479,1.989690e-01
3,tcp_sport,3305.758301,0.038360,1.887003e-02
4,ip_id,829.640015,0.009627,4.735776e-03
5,tcp_seq,718.773560,0.008341,4.102925e-03
6,tcp_dataofs,274.149597,0.003181,1.564909e-03
7,ip_len,79.276604,0.000920,4.525291e-04
8,tcp_flags.SN,68.066971,0.000790,3.885419e-04
9,tcp_flags.SPAU,9.213530,0.000107,5.259295e-05


ann


,variable,relative_importance,scaled_importance,percentage
0,tcp_sport,1.000000,1.000000,0.077655
1,tcp_flags.S,0.875147,0.875147,0.067959
2,tcp_dport,0.805115,0.805115,0.062521
3,tcp_ack,0.672796,0.672796,0.052246
4,tcp_window,0.662548,0.662548,0.051450
5,tcp_flags.SN,0.652960,0.652960,0.050705
6,ip_ttl,0.628964,0.628964,0.048842
7,tcp_dataofs,0.587567,0.587567,0.045627
8,udp_dport,0.548001,0.548001,0.042555
9,ip_proto,0.540191,0.540191,0.041948


In [106]:
f1 = [0.850425,0.961471,0.961583,0.958651,0.883111]

p = []
i = 0
for key in m_models:
    print(key)
    if key != 'bayes':
        # varimp.append(m_models[key].varimp())
        varimp = m_models[key].varimp(use_pandas=True)
        del varimp['percentage']
        del varimp['relative_importance']
        varimp['scaled_importance'] = varimp['scaled_importance'] * f1[i]
        p += varimp.values.tolist()
        i +=1 

p

bayes
glm
rf
gbm
xgboost
ann


[['tcp_dataofs', 0.850425],
 ['tcp_dport', 0.6280741259642519],
 ['ip_ttl', 0.5932394796759066],
 ['tcp_sport', 0.5813158652679632],
 ['ip_len', 0.46136627599678887],
 ['tcp_window', 0.41966711248843225],
 ['tcp_ack', 0.16183965139471138],
 ['udp_len', 0.14115908009279546],
 ['tcp_flags.S', 0.0911348482539641],
 ['ip_id', 0.04696691664851115],
 ['tcp_flags.SPAU', 0.03813714887908264],
 ['ip_tos', 0.030034261624263835],
 ['ip_proto', 0.014958895147332509],
 ['udp_sport', 0.013750813973228007],
 ['tcp_seq', 0.00944074979512456],
 ['tcp_flags.A', 0.0],
 ['tcp_flags.FA', 0.0],
 ['tcp_flags.PA', 0.0],
 ['tcp_flags.RA', 0.0],
 ['tcp_flags.SA', 0.0],
 ['tcp_flags.SN', 0.0],
 ['udp_dport', 0.0],
 ['icmp_unused', 0.0],
 ['tcp_flags', 0.961471],
 ['tcp_sport', 0.7585496479977055],
 ['tcp_dport', 0.5291078729146148],
 ['tcp_window', 0.5174000265799907],
 ['ip_len', 0.3302992066826553],
 ['tcp_dataofs', 0.279482519019999],
 ['ip_id', 0.2607718920364966],
 ['ip_ttl', 0.20974874374736874],
 ['tcp_se

In [107]:
a=pd.DataFrame(p) 
a
a.to_csv('a.csv',index=False)